## Step 4 Create Network Graph, Assign Link Costs, and Run BikewaySim

1. Process network spatial data into a routable network graph format
2. Reconcile networks into one through node and link overlap conflation
3. __Create final network graph and calculate link costs__
4. Create OD tables
5. Run BikewaySim

In [1]:
from pathlib import Path
import geopandas as gpd

In [2]:
from prepare_network import *

## Export Filepath

In [3]:
studyarea_name = 'bikewaysim'
export_fp = Path.home() / Path(f'Documents/NewBikewaySimData/{studyarea_name}')

## Link Costs Dictionary
Dict keys must correspond to column names in links GeoDataFrame.
The links cost funciton is off this format:
$$ linkcost = linkdistance * (1-\sum \beta x) $$

Negative attributes **decrease** impedance  
Positive attributes **increase** impedance

In [4]:
costs0 = {
    'bl':-0.05,
    'pbl':-0.95,
    'mu':-0.95,
    'below25':0,
    '25-30':0.30,
    'above30':1,
    '1laneper':0,
    '2to3lanesper':0.50,
    '4ormorelanesper':1,
    'wrongway':2       
    }

In [5]:
costs1 = {
    'bl':-0.05,
    'pbl':-0.5,
    'mu':-0.5,
    'below25':0,
    '25-30':0.30,
    'above30':1,
    '1laneper':0,
    '2to3lanesper':0.50,
    '4ormorelanesper':1,
    'wrongway':2       
    }

In [6]:
costs2 = {
    'bl':-0.05,
    'pbl':-0.75,#0.95,
    'mu':-0.75,
    'below25':0,
    '25-30':0.30,
    'above30':1,
    '1laneper':0,
    '2to3lanesper':0.50,
    '4ormorelanesper':1,
    'wrongway':2       
    }

In [7]:
cost_dicts = {
    'costs0':costs0,
    'costs1':costs1,
    'costs2':costs2
}

In [8]:
#%% TRB 2023 Code bring in links and nodes

#import links you want to use (these are the trb 2023 links)
links = gpd.read_file(export_fp / Path('reconciled_network.gpkg'),layer='here_trb')
nodes = gpd.read_file(export_fp / Path('reconciled_network.gpkg'),layer='here_trb_nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)

for key in cost_dicts.keys():

    #other costs to test
    links = link_costs(links, cost_dicts[key], key)

    #make fp organizing
    if not (export_fp / Path(key)).exists():
        (export_fp / Path(key)).mkdir()

    #where did link costs go up?
    up = links[links[key] > links['mins']]
    if up.shape[0] > 0:
        up.to_file(export_fp / Path(f'{key}/link_cost_changes.gpkg'),layer='increased')

    #where did they decrease?
    down = links[links[key] < links['mins']]
    if down.shape[0] > 0:
        down.to_file(export_fp / Path(f'{key}/link_cost_changes.gpkg'),layer='decreased')

    #where did they stay the same
    same = links[links[key] == links['mins']]
    if same.shape[0] > 0:
        same.to_file(export_fp / Path(f'{key}/link_cost_changes.gpkg'),layer='same')

#export
nodes.to_file(export_fp/Path('final_network.gpkg'),layer='nodes',driver='GPKG')
links.to_file(export_fp/Path('final_network.gpkg'),layer='links',driver='GPKG')

warning, more than one id column present


## Improvements

In [9]:
#make improvements
improvements = links.copy()
changes = '10TH ST NW'
#add mu
improvements.loc[improvements['ST_NAME']==changes,'mu'] = 1

#bring in new feature and old links
new = gpd.read_file(export_fp/Path('network_improvements.gpkg'),layer='new')
#make reverse
new = create_reverse_links(new)
new['dist'] = new.length
new['mins'] = new['dist'] / 5280 / 8 * 60

#add to network
improvements = improvements.append(new)

#redo costs
for key in cost_dicts.keys():
    #other costs to test
    improvements = link_costs(improvements, cost_dicts[key], key)

#export
improvements.to_file(export_fp/Path('final_network.gpkg'),layer='improved_links')


In [11]:
len(new)

10

,A,B,A_B,ST_NAME,AR_AUTO,below25,25-30,above30,1laneper,2to3lanesper,...,mu,pbl,bl,geometry,dist,mins,imp_factor,costs0,costs1,costs2
0,2044579814,2044579823,2044579814_2044579823,JOHN LEWIS FREEDOM PKWY NE,Y,0,0,1,0,1,...,0,0,0,"LINESTRING (2238293.048 1370792.171, 2238278.0...",709.32,1.01,2.5,2.525,2.525,2.525
2,2044579808,2044579814,2044579808_2044579814,JOHN LEWIS FREEDOM PKWY NE,Y,0,0,1,0,1,...,0,0,0,"LINESTRING (2238307.924 1370621.124, 2238293.0...",171.69,0.24,2.5,0.600,0.600,0.600
3,2044581591,2017261839,2044581591_2017261839,AVERY DR NE,Y,0,1,0,1,0,...,0,0,0,"LINESTRING (2233472.798 1381073.609, 2233463.8...",58.93,0.08,1.3,0.104,0.104,0.104
4,2017289087,2017289089,2017289087_2017289089,ANSLEY VILLA DR NE,Y,0,1,0,1,0,...,0,0,0,"LINESTRING (2233264.382 1384665.450, 2233148.9...",177.15,0.25,1.3,0.325,0.325,0.325
5,2017289089,2045981947,2017289089_2045981947,ANSLEY VILLA DR NE,Y,0,1,0,1,0,...,0,0,0,"LINESTRING (2233088.225 1384676.724, 2233048.7...",316.39,0.45,1.3,0.585,0.585,0.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2044584597,2044584616,2044584597_2044584616,morningside path,NaN,0,0,0,0,0,...,1,0,0,"LINESTRING (2238487.331 1383946.456, 2238975.3...",NaN,NaN,NaN,NaN,NaN,NaN
6,2044581836,2044581825,2044581836_2044581825,None,NaN,0,0,0,0,0,...,1,0,0,"LINESTRING (2235732.025 1379850.213, 2236280.8...",NaN,NaN,NaN,NaN,NaN,NaN
7,201157663548,2044581873,201157663548_2044581873,None,NaN,0,0,0,0,0,...,1,0,0,"LINESTRING (2236803.968 1380967.356, 2237216.7...",NaN,NaN,NaN,NaN,NaN,NaN
8,2044581950,201157663548,2044581950_201157663548,None,NaN,0,0,0,0,0,...,1,0,0,"LINESTRING (2237535.650 1381871.160, 2237636.1...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#%% NCST Project
road_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_links')
road_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_nodes')

bike_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_links')
bike_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_nodes')

roadbike_links = road_links.append(bike_links)
roadbike_nodes = road_nodes.append(bike_nodes).drop_duplicates()

roadbike_links.rename(columns={'osm_A':'A','osm_B':'B'},inplace=True)

links, nodes = prepare_network(roadbike_links,roadbike_nodes,link_costs=True)

print('exporting...')
links.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='links',driver='GPKG')
nodes.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='nodes',driver='GPKG')
print('done.')